In [1]:
transition_matrix = {
    'A': {'A': 0.1, 'B': 0.6, 'C': 0.3},
    'B': {'A': 0.4, 'B': 0.5, 'C': 0.1},
    'C': {'A': 0.2, 'B': 0.3, 'C': 0.5}
}

def display_transition_matrix(matrix):
    states = list(matrix.keys())
    print("Transition Matrix:")
    print("    " + "  ".join(states))
    for state in states:
        row = [f"{matrix[state].get(s, 0.0):.2f}" for s in states]
        print(f"{state} | " + "  ".join(row))
    print()

def display_state_transitions(matrix):
    print("State Transitions:")
    for state, transitions in matrix.items():
        print(f"From state {state}:")
        for next_state, prob in transitions.items():
            print(f"  to {next_state} with probability {prob:.2f}")
    print()

display_transition_matrix(transition_matrix)

display_state_transitions(transition_matrix)

Transition Matrix:
    A  B  C
A | 0.10  0.60  0.30
B | 0.40  0.50  0.10
C | 0.20  0.30  0.50

State Transitions:
From state A:
  to A with probability 0.10
  to B with probability 0.60
  to C with probability 0.30
From state B:
  to A with probability 0.40
  to B with probability 0.50
  to C with probability 0.10
From state C:
  to A with probability 0.20
  to B with probability 0.30
  to C with probability 0.50



In [2]:
import numpy as np

In [3]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))  # Element-wise computation of sigmoid

def sigmoid_derivative(x):
    return x * (1 - x)  # Derivative of sigmoid: f'(x) = f(x)*(1 - f(x))

In [4]:
# -------------------- MLP Class Definition -------------------- #
class MLP:
    def __init__(self, input_size, hidden_size, output_size):
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01
        self.b2 = np.zeros((1, output_size))

    def forward(self, X):
        self.Z1 = np.dot(X, self.W1) + self.b1
        self.A1 = sigmoid(self.Z1)
        self.Z2 = np.dot(self.A1, self.W2) + self.b2
        self.A2 = sigmoid(self.Z2)
        return self.A2

    def backward(self, X, y, output, learning_rate):
        error = y - output
        dZ2 = error * sigmoid_derivative(output)
        dW2 = np.dot(self.A1.T, dZ2)
        db2 = np.sum(dZ2, axis=0, keepdims=True)

        dA1 = np.dot(dZ2, self.W2.T)
        dZ1 = dA1 * sigmoid_derivative(self.A1)
        dW1 = np.dot(X.T, dZ1)
        db1 = np.sum(dZ1, axis=0, keepdims=True)

        self.W1 += learning_rate * dW1
        self.b1 += learning_rate * db1
        self.W2 += learning_rate * dW2
        self.b2 += learning_rate * db2

    def train(self, X, y, learning_rate=0.1, epochs=10000):
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output, learning_rate)

            if epoch % 1000 == 0:
                loss = np.mean(np.square(y - output))
                print(f"Epoch {epoch}, Loss: {loss}")

    def predict(self, X):
        return self.forward(X)

In [5]:
transition_matrix = {
    'A': {'A': 0.1, 'B': 0.6, 'C': 0.3},
    'B': {'A': 0.4, 'B': 0.5, 'C': 0.1},
    'C': {'A': 0.2, 'B': 0.3, 'C': 0.5}
}

states = ['A', 'B', 'C']
state_to_vector = {}
vector_to_state = {}
identity = np.eye(len(states))

for i, state in enumerate(states):
    state_to_vector[state] = identity[i]
    vector_to_state[i] = state 

In [6]:
def generate_sequence(start_state, length):
    sequence = [start_state]
    current_state = start_state
    for _ in range(length - 1):
        transitions = transition_matrix[current_state]
        next_states = list(transitions.keys())
        probabilities = list(transitions.values())
        next_state = np.random.choice(next_states, p=probabilities)
        sequence.append(next_state)
        current_state = next_state
    return sequence

In [7]:
training_inputs = []
training_outputs = []

num_sequences = 1000
sequence_length = 10

for _ in range(num_sequences):
    start_state = np.random.choice(states)
    seq = generate_sequence(start_state, sequence_length)
    for i in range(len(seq) - 1):
        input_state = seq[i] 
        next_state = seq[i + 1]
        training_inputs.append(state_to_vector[input_state])
        training_outputs.append(state_to_vector[next_state])

In [8]:

X_train = np.array(training_inputs)
y_train = np.array(training_outputs)

mlp = MLP(input_size=len(states), hidden_size=5, output_size=len(states))

mlp.train(X_train, y_train, learning_rate=0.0001, epochs=10000)


Epoch 0, Loss: 0.24906331488950625
Epoch 1000, Loss: 0.19755247530737124
Epoch 2000, Loss: 0.19312759177730218
Epoch 3000, Loss: 0.1930569330940943
Epoch 4000, Loss: 0.19305443920744625
Epoch 5000, Loss: 0.1930540207449535
Epoch 6000, Loss: 0.193053942238523
Epoch 7000, Loss: 0.19305392742787472
Epoch 8000, Loss: 0.19305392462984128
Epoch 9000, Loss: 0.19305392410091315


In [9]:
print("\nPredicted transition probabilities from each state:")
for state in states:
    input_vector = state_to_vector[state].reshape(1, -1)
    predicted_output = mlp.predict(input_vector)
    print(f"From state {state}:")
    for i, next_state in vector_to_state.items():
        print(f"  To {next_state}: {predicted_output[0][i]:.3f}")


Predicted transition probabilities from each state:
From state A:
  To A: 0.100
  To B: 0.596
  To C: 0.304
From state B:
  To A: 0.390
  To B: 0.512
  To C: 0.098
From state C:
  To A: 0.200
  To B: 0.302
  To C: 0.499
